In [ ]:
!pip install pandas
!pip install numpy
# !pip install deap
# !pip install sklearn

In [ ]:
# IMPORTING LIBS

import pandas as pd
import numpy as np
import random
# from deap import base, creator, tools
# from sklearn.preprocessing import MinMaxScaler


In [ ]:
# IMPORT FILE (BC OF GOOGLE COLLAB)
from google.colab import files
  
# select all 4 datasets called data1, data2, data3, data4
uploaded = files.upload()

Saving data1.csv to data1.csv
Saving data2.csv to data2.csv
Saving data3.csv to data3.csv
Saving data4.csv to data4.csv


In [ ]:
# GET DATA FROM CSV
import io
  
df1 = pd.read_csv(io.BytesIO(uploaded['data1.csv']))
# WORK WITH ONLY ONE DATASET THEN SCALE UP AND DO MERGING NORMALIZATION LATER XD
# df2 = pd.read_csv(io.BytesIO(uploaded['data2.csv']))
# df3 = pd.read_csv(io.BytesIO(uploaded['data3.csv']))
# df4 = pd.read_csv(io.BytesIO(uploaded['data4.csv']))

In [ ]:
# GET THE 25 PLAYERS THAT PLAYED THE MOST

sorted_df1 = df1.sort_values(by = "GP", ascending = False).head(25)
# sorted_df2 = df2.sort_values(by = "GP", ascending = False).head(25)
# sorted_df3 = df3.sort_values(by = "GP", ascending = False).head(25)
# sorted_df4 = df4.sort_values(by = "GP", ascending = False).head(25)

In [ ]:
# MERGING DATA FRAMES INTO ONE

# # merged_df = pd.merge(sorted_df1, sorted_df2, on="Player")
# merged_df = pd.concat([sorted_df1, sorted_df2]).groupby(['Player']).sum().reset_index()
# merged_df

In [ ]:
# DEFINING FUNCTIONS FOR HILL CLIMBING SOLUTION

def create_solution():
  pass

def expand_neighborhood():
  pass

def target_fn():
  pass

  

In [ ]:
# DEFINING GAMEPLAY TYPES

# type calculation

# Agressive
# very high kills, moderate deaths, low assists, moderate farm, very high DPM, low WPM

# Farmer
# low kills, low deaths, moderate assists, very high farm, low DPM, moderate WPM

# Support
# very low kills, moderate deaths, high assists, very low farm, low DPM, very high WPM

# normalized_data = sorted_df1.copy()
df_min_max_scaled = sorted_df1.copy()
# apply normalization techniques
# for column in normalized_data.columns:
#     if normalized_data[column].dtypes == "int64" or normalized_data[column].dtypes == "float64":
#       normalized_data[column] = normalized_data[column]  / normalized_data[column].abs().max()

for column in df_min_max_scaled.columns:
  if (normalized_data[column].dtypes == "int64" or normalized_data[column].dtypes == "float64") and column != "GP":
    df_min_max_scaled[column] = (df_min_max_scaled[column] - df_min_max_scaled[column].min()) / (df_min_max_scaled[column].max() - df_min_max_scaled[column].min())    
  

normalized_data = df_min_max_scaled.copy()


normalized_data['agressive'] = \
  normalized_data.apply(lambda row: abs(((row['K'] * 5) + ((row['D'] * 3)) + (row['A'] * 2) + (row['CSPM'] * 3) + (row['DPM'] * 5) + (row['WPM'] * 2)) / row['GP']), axis=1)

normalized_data['farmer'] = \
  normalized_data.apply(lambda row: abs(((row['K'] * 2) + ((row['D'] * 2)) + (row['A'] * 3) + (row['CSPM'] * 5) + (row['DPM'] * 2) + (row['WPM'] * 3)) / row['GP']), axis=1)

normalized_data['support'] = \
  normalized_data.apply(lambda row: abs(((row['K'] * 1) + ((row['D'] * 3)) + (row['A'] * 4) + (row['CSPM'] * 1) + (row['DPM'] * 2) + (row['WPM'] * 5)) / row['GP']), axis=1)

normalized_data['agressive'] = (normalized_data['agressive'] - normalized_data['agressive'].min()) / (normalized_data['agressive'].max() - normalized_data['agressive'].min())   
normalized_data['farmer'] = (normalized_data['farmer'] - normalized_data['farmer'].min()) / (normalized_data['farmer'].max() - normalized_data['farmer'].min())   
normalized_data['support'] = (normalized_data['support'] - normalized_data['support'].min()) / (normalized_data['support'].max() - normalized_data['support'].min())   

# normalized_data['farmer'] = normalized_data['farmer']  / normalized_data['farmer'].abs().max()
# normalized_data['support'] = normalized_data['support']  / normalized_data['support'].abs().max()

agressive_df = normalized_data.sort_values(by="agressive", ascending=False)
farmer_df = normalized_data.sort_values(by="farmer", ascending=False)
support_df = normalized_data.sort_values(by="support", ascending=False)

In [ ]:
# BEST TEAM FORMATION
# IDX ARE TOP, JG, MID, ADC, SUP
agresive_team = [1, 1, 1, 2, 3]
farmer_team   = [3, 1, 2, 2, 3]
support_team  = [3, 3, 2, 2, 3]